In [2]:
from google.cloud import storage


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    print(
        f"Uploading file {source_file_name} to {destination_blob_name} in bucket {bucket_name}."
    )

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    print(f"Uploading file {source_file_name} to {destination_blob_name} in bucket {bucket_name}.")

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    print(f"Generation match precondition: {generation_match_precondition}")

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )


In [3]:
upload_blob(
    "musp",
    "/home/aiwaki/musp/api/tmp/67b121dc-2ecd-48e5-9c09-888b08ec1433/separated/htdemucs/source/vocals.wav",
    "kunugi/test.wav",
)

Uploading file /home/aiwaki/musp/api/tmp/67b121dc-2ecd-48e5-9c09-888b08ec1433/separated/htdemucs/source/vocals.wav to kunugi/test.wav in bucket musp.
Uploading file /home/aiwaki/musp/api/tmp/67b121dc-2ecd-48e5-9c09-888b08ec1433/separated/htdemucs/source/vocals.wav to kunugi/test.wav in bucket musp.
Generation match precondition: 0
File /home/aiwaki/musp/api/tmp/67b121dc-2ecd-48e5-9c09-888b08ec1433/separated/htdemucs/source/vocals.wav uploaded to kunugi/test.wav.
